In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

import torch
import numpy as np

from dataset import SliceDataModule
from common.utils import save_reconstructions
from collections import defaultdict
from tqdm import tqdm
from pathlib import Path

In [4]:
HASH = "2afc06da58224c798b8f8c9969e696eb"
SAVE_DIR = f"reconstructions/{HASH}"

In [5]:
from models import VarNetLogisticUnetSensFixOL as Model

checkpoint = torch.load(f"mlruns/0/{HASH}/artifacts/checkpoints/latest_checkpoint.pth")
model = Model(**checkpoint["hyper_parameters"]["init_args"])
model.load_state_dict(torch.load(f"mlruns/0/{HASH}/artifacts/checkpoints/latest_checkpoint.pth")["state_dict"])
model = model.cuda()

In [6]:
dm = SliceDataModule(root="/home/Data")
dm.setup("test")
dm.setup("predict")

In [7]:
model.eval()
reconstructions = defaultdict(dict)
print("Reconstructing public leaderboard...")
with torch.no_grad():
    for (mask, kspace, target, maximum, fnames, slices) in tqdm(dm.test_dataloader()):
        output = model(kspace.cuda(non_blocking=True), mask.cuda(non_blocking=True))
        break
        output = model.image_space_crop(output)
        for i in range(output.shape[0]):
            reconstructions[fnames[i]][slices[i]] = output[i].cpu().numpy()
return


for fname in reconstructions:
    reconstructions[fname] = np.stack([reconstructions[fname][slice] for slice in sorted(reconstructions[fname])])
print("Saving reconstructions of public leaderboard...")
save_reconstructions(reconstructions, Path(f"{SAVE_DIR}/public"))


model.eval()
reconstructions = defaultdict(dict)
print("Reconstructing private leaderboard...")
with torch.no_grad():
    for (mask, kspace, target, maximum, fnames, slices) in tqdm(dm.predict_dataloader()):
        output = model(kspace.cuda(non_blocking=True), mask.cuda(non_blocking=True))
        output = model.image_space_crop(output)
        for i in range(output.shape[0]):
            reconstructions[fnames[i]][slices[i]] = output[i].cpu().numpy()
    
for fname in reconstructions:
    reconstructions[fname] = np.stack([reconstructions[fname][slice] for slice in sorted(reconstructions[fname])])
print("Saving reconstructions of private leaderboard...")
save_reconstructions(reconstructions, Path(f"{SAVE_DIR}/private"))


Reconstructing public leaderboard...


  0%|          | 0/984 [00:00<?, ?it/s]

  0%|          | 0/984 [00:00<?, ?it/s]


SyntaxError: 'return' outside function (4052954687.py, line 11)

In [9]:
output.shape

torch.Size([1, 768, 396])

In [9]:
!sh leaderboard_eval.sh {HASH}

Model: 2afc06da58224c798b8f8c9969e696eb

Leaderboard SSIM : 0.9619
========== Details ==========
Leaderboard SSIM (public): 0.9701
Leaderboard SSIM (private): 0.9537
